# LangChain Agent with AWS Bedrock and MCP Servers Demo

This notebook demonstrates a LangChain agent that uses AWS Bedrock for LLM capabilities and integrates with MCP (Model Context Protocol) servers for external tool functionality.

In [1]:
# Install required packages
%pip install langchain langchain-aws langchain-community boto3 python-dotenv requests

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import boto3
import requests
from dotenv import load_dotenv
from langchain_aws import ChatBedrock
from langchain.agents import create_react_agent, AgentExecutor
from langchain.tools import Tool
from langchain.prompts import PromptTemplate
from langchain_core.messages import HumanMessage

# Load environment variables
load_dotenv()

print("Packages imported successfully!")

Packages imported successfully!


In [3]:
# Set up LangSmith tracing (optional)
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY", "")
os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGCHAIN_PROJECT", "mcp-agent-demo")

print(f"LangSmith tracing enabled: {os.getenv('LANGCHAIN_TRACING_V2')}")
print(f"Project: {os.getenv('LANGCHAIN_PROJECT')}")

LangSmith tracing enabled: true
Project: mcp-agent-demo


In [4]:
# Initialize AWS Bedrock LLM
bedrock_llm = ChatBedrock(
    model_id="anthropic.claude-3-sonnet-20240229-v1:0",
    region_name=os.getenv("AWS_REGION", "us-east-1"),
    model_kwargs={
        "max_tokens": 1000,
        "temperature": 0.1
    }
)

print("Bedrock LLM initialized successfully!")

# Test the LLM
test_response = bedrock_llm.invoke([HumanMessage(content="Hello! Can you confirm you're working?")])
print(f"LLM Test Response: {test_response.content}")

Bedrock LLM initialized successfully!


/Users/parampatel/Desktop/cursor_demo/agentMCPDemo/.venv/lib/python3.13/site-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(
Failed to multipart ingest runs: langsmith.utils.LangSmithAuthError: Authentication failed for https://api.smith.langchain.com/runs/multipart. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Unauthorized"}\n')trace=c47c9a1e-4d0b-478d-a290-43e43b8ab711,id=c47c9a1e-4d0b-478d-a290-43e43b8ab711


LLM Test Response: Yes, I am working and ready to assist you. How can I help you today?


Failed to send compressed multipart ingest: langsmith.utils.LangSmithAuthError: Authentication failed for https://api.smith.langchain.com/runs/multipart. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Unauthorized"}\n')trace=c47c9a1e-4d0b-478d-a290-43e43b8ab711,id=c47c9a1e-4d0b-478d-a290-43e43b8ab711
Failed to send compressed multipart ingest: langsmith.utils.LangSmithAuthError: Authentication failed for https://api.smith.langchain.com/runs/multipart. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Unauthorized"}\n')trace=390caad3-9281-4630-b513-4f8a2a4e37e4,id=390caad3-9281-4630-b513-4f8a2a4e37e4; trace=390caad3-9281-4630-b513-4f8a2a4e37e4,id=f6b034bf-79a7-413c-82cf-e8a791c28823; trace=390caad3-9281-4630-b513-4f8a2a4e37e4,id=094dcacd-241c-4769-acf6-a348f8b755bf; trace=390caad3-9281-4630-b513-4f8a2a4e37e4,id=0169d8b3-99cf-4d94-a6f2-355d93967f28; trace=390caad3-

In [5]:
# Define MCP Server URLs
CALCULATOR_URL = os.getenv("MCP_CALCULATOR_URL", "http://localhost:8001")
WEATHER_URL = os.getenv("MCP_WEATHER_URL", "http://localhost:8002")

print(f"Calculator MCP Server: {CALCULATOR_URL}")
print(f"Weather MCP Server: {WEATHER_URL}")

Calculator MCP Server: http://localhost:8001
Weather MCP Server: http://localhost:8002


In [6]:
# Define tools to interact with MCP servers
import re

def call_calculator(expression: str) -> str:
    """Call the calculator MCP server to evaluate a mathematical expression."""
    try:
        expression = expression.strip()
        
        # Handle complex expressions like "25 * 4 + 10" by evaluating step by step
        if "+" in expression and "*" in expression:
            # Find multiplication first (order of operations)
            mult_match = re.search(r'(\d+(?:\.\d+)?)\s*\*\s*(\d+(?:\.\d+)?)', expression)
            if mult_match:
                # Calculate multiplication first
                a, b = float(mult_match.group(1)), float(mult_match.group(2))
                mult_response = requests.post(
                    f"{CALCULATOR_URL}/calculate",
                    json={"operation": "multiply", "a": a, "b": b},
                    timeout=10
                )
                if mult_response.status_code == 200:
                    mult_result = mult_response.json().get("result")
                    # Replace the multiplication part with the result
                    remaining = expression.replace(mult_match.group(0), str(mult_result))
                    
                    # Now handle the addition
                    add_match = re.match(r'(\d+(?:\.\d+)?)\s*\+\s*(\d+(?:\.\d+)?)', remaining)
                    if add_match:
                        c, d = float(add_match.group(1)), float(add_match.group(2))
                        add_response = requests.post(
                            f"{CALCULATOR_URL}/calculate",
                            json={"operation": "add", "a": c, "b": d},
                            timeout=10
                        )
                        if add_response.status_code == 200:
                            final_result = add_response.json().get("result")
                            return f"{expression} = {final_result}"
        
        # Handle basic two-operand expressions like "25 * 4"
        match = re.match(r'(\d+(?:\.\d+)?)\s*([+\-*/])\s*(\d+(?:\.\d+)?)', expression)
        if match:
            a, op, b = match.groups()
            a, b = float(a), float(b)
            
            # Map operators to operation names
            op_map = {'+': 'add', '-': 'subtract', '*': 'multiply', '/': 'divide'}
            operation = op_map.get(op)
            
            if operation:
                response = requests.post(
                    f"{CALCULATOR_URL}/calculate",
                    json={"operation": operation, "a": a, "b": b},
                    timeout=10
                )
                if response.status_code == 200:
                    result = response.json().get("result")
                    return f"{expression} = {result}"
                else:
                    return f"Calculator server error: {response.status_code}"
        
        return f"Error: Cannot parse expression '{expression}'. Please use simple format like '25 * 4' or '25 * 4 + 10'"
        
    except Exception as e:
        return f"Failed to connect to calculator server: {str(e)}"

def get_weather(location: str) -> str:
    """Get weather information for a given location from the weather MCP server."""
    try:
        # Fixed: Use correct endpoint, POST method, and JSON body format
        response = requests.post(
            f"{WEATHER_URL}/get_weather",
            json={"city": location},
            timeout=10
        )
        if response.status_code == 200:
            weather_data = response.json()
            return f"Weather in {weather_data.get('city', location)}: {weather_data.get('description', 'N/A')}, Temperature: {weather_data.get('temperature', 'N/A')}°C, Humidity: {weather_data.get('humidity', 'N/A')}%"
        else:
            return f"Weather server error: {response.status_code}"
    except Exception as e:
        return f"Failed to connect to weather server: {str(e)}"

# Create LangChain tools
calculator_tool = Tool(
    name="Calculator",
    func=call_calculator,
    description="Use this tool to perform mathematical calculations. Input should be a mathematical expression as a string."
)

weather_tool = Tool(
    name="Weather",
    func=get_weather,
    description="Use this tool to get weather information for a location. Input should be a location name as a string."
)

tools = [calculator_tool, weather_tool]
print(f"Created {len(tools)} tools for the agent")

Created 2 tools for the agent


In [7]:
# Create the agent prompt template
prompt_template = PromptTemplate.from_template(
    """
You are a helpful AI assistant with access to external tools via MCP servers.

You have access to the following tools:
{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought: {agent_scratchpad}
"""
)

print("Agent prompt template created")

Agent prompt template created


In [8]:
# Create and initialize the LangChain agent
agent = create_react_agent(
    llm=bedrock_llm,
    tools=tools,
    prompt=prompt_template
)

agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    max_iterations=5,
    handle_parsing_errors=True
)

print("LangChain agent created and ready!")

LangChain agent created and ready!


In [9]:
# Test the agent with example prompts
print("=== Testing Calculator Tool ===")
result1 = agent_executor.invoke({"input": "What is 25 * 4 + 10?"})
print(f"Result: {result1['output']}")
print()

print("=== Testing Weather Tool ===")
result2 = agent_executor.invoke({"input": "What's the weather like in New York?"})
print(f"Result: {result2['output']}")
print()

print("=== Testing Combined Usage ===")
result3 = agent_executor.invoke({
    "input": "Calculate 15 * 8, then tell me the weather in London"
})
print(f"Result: {result3['output']}")

=== Testing Calculator Tool ===


> Entering new AgentExecutor chain...
Thought: To solve this, I need to perform the mathematical calculation 25 * 4 + 10.
Action: Calculator
Action Input: 25 * 4 + 1025 * 4 + 10 = 110.0Question: What is 25 * 4 + 10?
Thought: To solve this, I need to perform the mathematical calculation 25 * 4 + 10.
Action: Calculator
Action Input: 25 * 4 + 1025 * 4 + 10 = 110.0Thought: I now know the final answer
Final Answer: 110

> Finished chain.
Result: 110

=== Testing Weather Tool ===


> Entering new AgentExecutor chain...
Thought: To find out the weather in New York, I should use the Weather tool.

Action: Weather
Action Input: New York
Weather in New York
: Random weather condition, Temperature: 30.4°C, Humidity: 49%Question: What's the weather like in New York?
Thought: To find out the weather in New York, I should use the Weather tool.

Action: Weather
Action Input: New York
Weather in New York
: Random weather condition, Temperature: 31.1°C, Humidity: 63%Th

## How to Use This Demo

1. **Prerequisites**: 
   - Make sure your MCP servers are running (use `docker-compose up` in the project root)
   - Set up your AWS credentials and environment variables
   - Configure LangSmith if you want tracing

2. **Run the cells above** to initialize the agent

3. **Try your own prompts** in the cell below:


In [ ]:
# Try your own prompts here!
custom_prompt = "Calculate the square root of 144 and tell me the weather in Paris"

result = agent_executor.invoke({"input": custom_prompt})
print(f"Result: {result['output']}")

## Monitoring and Tracing

If you have LangSmith configured, you can view detailed traces of the agent's execution at:
https://smith.langchain.com/

The traces will show:
- LLM calls to Bedrock
- Tool invocations to MCP servers
- Agent reasoning steps
- Performance metrics